Calculate current from voltage and line parameters

In [1]:
import numpy as np

bdata = np.genfromtxt("System_data/bdata.csv", delimiter=",")


In [2]:
vr = np.genfromtxt("dataset_pf_corrected/V_real.csv", delimiter=",")
vi = np.genfromtxt("dataset_pf_corrected/V_imag.csv", delimiter=",")

cfr = np.genfromtxt("dataset_pf_corrected/CF_real.csv", delimiter=",")
cfi = np.genfromtxt("dataset_pf_corrected/CF_imag.csv", delimiter=",")

ctr = np.genfromtxt("dataset_pf_corrected/CT_real.csv", delimiter=",")
cti = np.genfromtxt("dataset_pf_corrected/CT_imag.csv", delimiter=",")


In [3]:
cfr_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cfr_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

cfi_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cfi_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

ctr_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
ctr_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

cti_vr_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))
cti_vi_matrix  = np.zeros((vr.shape[1], cfr.shape[1]))

for idx,branch in enumerate(bdata):
    fbus = int(branch[0])-1
    tbus = int(branch[1])-1
    R = branch[2]
    X = branch[3]
    B = branch[4]
    TAU = branch[5]

    AR = R/(TAU**2 *(R**2 + X**2))
    AI = B/(2*TAU**2) - X/(TAU**2 *(R**2 + X**2))

    BR = -R/(TAU*(R**2 + X**2))
    BI = X/(TAU*(R**2 + X**2))

    DR = R/(R**2 + X**2)
    DI = B/2 - X/(R**2 + X**2)


    cfr_vr_matrix[fbus,idx] = AR
    cfr_vr_matrix[tbus,idx] = BR

    cfr_vi_matrix[fbus,idx] = -AI
    cfr_vi_matrix[tbus,idx] = -BI

    cfi_vr_matrix[fbus,idx] = AI
    cfi_vr_matrix[tbus,idx] = BI

    cfi_vi_matrix[fbus,idx] = AR
    cfi_vi_matrix[tbus,idx] = BR



    ctr_vr_matrix[fbus,idx] = BR
    ctr_vr_matrix[tbus,idx] = DR

    ctr_vi_matrix[fbus,idx] = -BI
    ctr_vi_matrix[tbus,idx] = -DI

    cti_vr_matrix[fbus,idx] = BI
    cti_vr_matrix[tbus,idx] = DI

    cti_vi_matrix[fbus,idx] = BR
    cti_vi_matrix[tbus,idx] = DR


In [4]:
vr_q = vr[0,:]
vi_q = vi[0,:]

cfr_a = cfr[0,:]
cfi_a = cfi[0,:]

ctr_a = ctr[0,:]
cti_a = cti[0,:]

from variable_space import current_from_voltage

a,b,c,d = current_from_voltage(vr_q, vi_q, bdata)

# ctr_hat = np.matmul(vr_q.T, ctr_vr_matrix) + np.matmul(vi_q.T, ctr_vi_matrix)
print(np.mean(a - cfr_a))

-1.2504441739823733e-15


In [5]:
a = np.array([1,2])
b= np.array([3,4])

a*b

array([3, 8])

For every branch,
$$P_f = v_f^r i^r_f + v^i_f i^i_f$$
$$P_t = v_t^r i^r_t + v^i_t i^i_t$$


$$P_f = (A V_r)(B C^f_r)+ (C V_i)(D C^f_i)$$ 

In [6]:
pf_v_matrix = np.zeros((vr.shape[1], cfr.shape[1]))
pt_v_matrix = np.zeros((vr.shape[1], cfr.shape[1]))
for idx, branch in enumerate(bdata):
    fbus = int(branch[0])-1
    tbus = int(branch[1])-1
    pf_v_matrix[fbus,idx] = 1
    pt_v_matrix[tbus, idx] = 1

 

In [7]:
vr_test = vr[0,:]
vi_test = vi[0,:]
cfr_test = cfr[0,:]
cfi_test = cfi[0,:]
ctr_test = ctr[0,:]
cti_test = cti[0,:]

pf_hat = np.matmul(vr_test, pf_v_matrix)* cfr_test*100 + np.matmul(vi_test, pf_v_matrix)*cfi_test*100
pt_hat = np.matmul(vr_test, pt_v_matrix)* ctr_test*100 + np.matmul(vi_test, pt_v_matrix)*cti_test*100


In [8]:
np.savetxt("System_data\pf_v_matrix.csv", pf_v_matrix, delimiter= ',')
np.savetxt("System_data\pt_v_matrix.csv", pt_v_matrix, delimiter= ',')


In [9]:
from final_layer_weights import final_layer_weights_reordered

weights = final_layer_weights_reordered()
weights.shape

(372, 118)

In [10]:
Pflow = np.hstack([pf_hat, pt_hat])
Pflow.shape

(372,)

In [11]:
Pinj = np.matmul(Pflow.T,weights )

Pinj

array([-2.24193705e+01, -1.70820000e+01, -3.55250000e+01, -3.84629670e+01,
        2.52333621e-09, -4.43193430e+01, -2.44660000e+01, -4.07019733e+01,
        8.66800178e-08,  4.06303553e+02, -8.22529999e+01,  4.59903053e+01,
       -2.65819999e+01, -1.25009999e+01, -7.60186706e+01, -2.66069998e+01,
       -9.37179805e+00, -3.70419368e+01, -2.43962574e+01, -1.87399999e+01,
       -1.54240000e+01, -1.00230001e+01, -9.39090007e+00, -1.76559655e+01,
        1.96016013e+02,  2.82850898e+02, -6.49648946e+01, -1.87249999e+01,
       -3.34179999e+01,  6.95595020e-07, -2.07376475e+01, -2.73405942e+01,
       -2.10149997e+01, -5.08450941e+01, -4.15669999e+01, -1.29512536e+01,
        3.10728183e-07,  3.01105764e-07, -2.40760000e+01, -3.31301621e+01,
       -2.95310000e+01, -5.20501223e+01, -1.65240000e+01, -1.73600000e+01,
       -6.10600000e+01, -1.01179725e+01, -3.52550000e+01, -9.40450000e+00,
        1.01115110e+02, -1.72150000e+01, -2.62490000e+01, -1.42480000e+01,
       -2.00660000e+01, -